In [1]:
#Training various popular neural networks (Resnet, VGGNet,Inception V3 etc) on custom Dataset.
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
# Choose the model you want to train (ResNet50, VGG16, or InceptionV3)
model_name = "ResNet50"  # Change this to VGG16 or InceptionV3 for other models

In [4]:
if model_name == "ResNet50":
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
elif model_name == "VGG16":
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
elif model_name == "InceptionV3":
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
else:
    raise ValueError("Invalid model name. Choose 'ResNet50', 'VGG16', or 'InceptionV3'.")

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

94765736/94765736 [==============================] - 72s 1us/step


In [5]:
# Add a global average pooling layer and a dense output layer for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(10, activation='softmax')(x)

In [6]:
# Create the model with the custom output layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [7]:
# Train the model
epochs = 10
batch_size = 32
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

Epoch 1/10


2023-07-28 23:41:20.352799: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 144/1563 [=>............................] - ETA: 1:44 - loss: 2.3793 - accuracy: 0.3444

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f'Test accuracy: {test_accuracy:.4f}')